In [1]:
#https://www.datacamp.com/community/tutorials/recommender-systems-python?utm_source=adwords_ppc&utm_campaignid=1455363063&utm_adgroupid=65083631748&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=332602034361&utm_targetid=aud-763347114660:dsa-473406569915&utm_loc_interest_ms=&utm_loc_physical_ms=1012760&gclid=CjwKCAjw4pT1BRBUEiwAm5QuR4fS-LTk9aJMxgVtVdomGDfamTTV9bLHZEI1_maU9Mtmzd4DhibudRoCz-sQAvD_BwE
import pandas as pd

In [2]:
data = pd.read_excel (r'./data2.xlsx')
data.head()

,Unnamed: 0,Userid,UUID,Version,CpuHertz,CPU_0,CPU_1,CPU_2,CPU_3,Total_CPU,...,tot_irq,tot_softirq,ctxt,btime,processes,procs_running,procs_blocked,connectedWifi_SSID,connectedWifi_Level,version_Battery_level
0,0,97bb95f55a,1462793200348,2.3.1,1.7 GHz,45.833336,52.380955,80.000000,61.904762,60.029762,...,3118,1950319,1331498712,1462388871,8384093,5,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_61.0
1,2,97bb95f55a,1462793213783,2.3.1,1.7 GHz,72.727270,7.692308,78.947370,4.761905,41.032210,...,3118,1950496,1331588413,1462388871,8384543,3,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_61.0
2,12,97bb95f55a,1462793264526,2.3.1,1.7 GHz,48.275860,68.181816,90.909096,61.904762,67.317880,...,3118,1950735,1331726828,1462388871,8385541,4,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_61.0
3,13,97bb95f55a,1462793269775,2.3.1,1.7 GHz,33.333336,83.333330,23.809525,4.761905,36.309525,...,3118,1950799,1331764427,1462388871,8385720,2,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_61.0
4,14,97bb95f55a,1462793276167,2.3.1,300 MHz,33.333336,25.000000,13.636364,0.000000,17.992424,...,3118,1950902,1331799034,1462388871,8386036,1,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_61.0


In [3]:
data['version_Battery_level']

0        2.3.1_61.0
1        2.3.1_61.0
2        2.3.1_61.0
3        2.3.1_61.0
4        2.3.1_61.0
            ...    
19995    2.3.2_83.0
19996    2.3.2_83.0
19997    2.3.2_83.0
19998    2.3.2_83.0
19999    2.3.2_82.0
Name: version_Battery_level, Length: 20000, dtype: object

In [4]:
data2 = data

In [5]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
data2['version_Battery_level'] = data2['version_Battery_level'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(data2['version_Battery_level'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape


(20000, 109)

In [6]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(20000, 20000)

In [7]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(data2.index, index=data2['version_Battery_level']).drop_duplicates()

In [8]:
indices['2.3.1_61.0']

2.3.1_61.0        0
2.3.1_61.0        1
2.3.1_61.0        2
2.3.1_61.0        3
2.3.1_61.0        4
              ...  
2.3.1_61.0    16474
2.3.1_61.0    16475
2.3.1_61.0    16476
2.3.1_61.0    16477
2.3.1_61.0    16478
Length: 211, dtype: int64

In [20]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(version,Battery_level, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    version_Battery_level = version+'_'+str(Battery_level)
    idx = indices[version_Battery_level]
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    #print(sim_scores)
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: max(x[1]), reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return data2.iloc[movie_indices]

In [21]:
get_recommendations('2.3.1',61.0)

,Unnamed: 0,Userid,UUID,Version,CpuHertz,CPU_0,CPU_1,CPU_2,CPU_3,Total_CPU,...,tot_irq,tot_softirq,ctxt,btime,processes,procs_running,procs_blocked,connectedWifi_SSID,connectedWifi_Level,version_Battery_level
1,2,97bb95f55a,1462793213783,2.3.1,1.7 GHz,72.727270,7.692308,78.947370,4.761905,41.032210,...,3118,1950496,1331588413,1462388871,8384543,3,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_61.0
2,12,97bb95f55a,1462793264526,2.3.1,1.7 GHz,48.275860,68.181816,90.909096,61.904762,67.317880,...,3118,1950735,1331726828,1462388871,8385541,4,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_61.0
3,13,97bb95f55a,1462793269775,2.3.1,1.7 GHz,33.333336,83.333330,23.809525,4.761905,36.309525,...,3118,1950799,1331764427,1462388871,8385720,2,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_61.0
4,14,97bb95f55a,1462793276167,2.3.1,300 MHz,33.333336,25.000000,13.636364,0.000000,17.992424,...,3118,1950902,1331799034,1462388871,8386036,1,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_61.0
5,16,97bb95f55a,1462793286417,2.3.1,1.7 GHz,94.736850,100.000000,4.545454,25.000000,56.070576,...,3118,1950952,1331828941,1462388871,8386247,3,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_60.0
6,22,97bb95f55a,1462793316890,2.3.1,1.7 GHz,42.857143,50.000000,10.526316,35.000000,34.595863,...,3118,1951071,1331911110,1462388871,8386872,2,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_60.0
7,23,97bb95f55a,1462793322078,2.3.1,1.7 GHz,3.846154,95.000000,100.000000,60.000004,64.711540,...,3118,1951112,1331929070,1462388871,8386945,4,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_60.0
8,24,97bb95f55a,1462793327233,2.3.1,1.7 GHz,7.407407,100.000000,95.454544,100.000000,75.715485,...,3118,1951165,1331951773,1462388871,8387019,5,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_60.0
9,28,97bb95f55a,1462793348748,2.3.1,1.7 GHz,50.000000,47.368423,68.181816,0.000000,41.387558,...,3119,1951394,1332052836,1462388871,8387692,3,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_60.0
10,34,97bb95f55a,1462793379130,2.3.1,1.7 GHz,8.695652,0.000000,100.000000,0.000000,27.173912,...,3119,1951533,1332122334,1462388871,8388257,4,0,0x3352A141D7A60F64879A4235D387F36B,-127,2.3.1_60.0
